In [ ]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt, animation, patches
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import openpyxl

import ephem
from datetime import datetime, timezone, timedelta

from skyfield import almanac
from skyfield.api import load
from shapely.geometry import LineString, Polygon


In [ ]:
input_file = "/Users/laytonrosenfeld/Desktop/Evolved/ADP electricity sample.xlsx"
lat_long_file = "/Users/laytonrosenfeld/Desktop/Evolved/zone_lat_lon.csv"
shapefile_path = "/Users/laytonrosenfeld/Desktop/Evolved/adp_regions/adp_regions.shp"
save_path = "/Users/laytonrosenfeld/Desktop/Evolved/animations"

interpolate_minutes = 15
month = 7
day = 27
year = 2011
date = f'{year}-{month:02d}-{day:02d}'
frames_per_second=2
scenario = 'central'


In [ ]:
adp_data_to = pd.read_excel(input_file, sheet_name=0)
adp_data_from = pd.read_excel(input_file, sheet_name=1)
lat_long_info = pd.read_csv(lat_long_file)

In [ ]:
transmission = adp_data_to[adp_data_to['From||Type'] == 'transmission']

def extract_zones(row):
    locations = row['From||Outputs Group Aggregate'].split('||')
    destination = row['Zone']
    source = next(loc for loc in locations if loc != destination)
    return pd.Series([source, destination])


def fill_missing_hours(transmission):
    df = transmission.copy()
    df['date'] = df['Weather Datetime'].dt.date  # Extract date for grouping
    df = df.set_index('Weather Datetime')  # Set index for resampling

    filled_results = []  

    for (source, destination, date), group in df.groupby(['Source', 'Destination', 'date']):
        full_range = pd.date_range(start=date, periods=24, freq='h')  # Generate all 24 hours
        group_resampled = group.reindex(full_range).fillna({'Value': 0})  # Fill missing hours with 0
        group_resampled[['Source', 'Destination']] = source, destination  # Preserve categorical data
        group_resampled = group_resampled.reset_index().rename(columns={'index': 'Weather Datetime'})
        filled_results.append(group_resampled)

    filled_df = pd.concat(filled_results, ignore_index=True)
    return filled_df



def interpolate_transmission(transmission):
    df = transmission.copy()  # Avoid modifying the original DataFrame
    df['date'] = df['Weather Datetime'].dt.date  # Extract date (day-level grouping)
    df = df.set_index('Weather Datetime')  # Set index for resampling

    interpolated_results = []  # List to store results

    for (source, destination, date), group in df.groupby(['Source', 'Destination', 'date']):
        group_resampled = group.resample('15min').interpolate(method='linear')  # Interpolate numeric columns
        group_resampled = group_resampled.ffill()  # Forward fill non-numeric columns
        group_resampled = group_resampled.reset_index()  # Reset index for merging later
                
        interpolated_results.append(group_resampled)  # Store result

    interpolated_df = pd.concat(interpolated_results, ignore_index=True)
    return interpolated_df

def compute_net_flows(transmission):
    net_flow_data = []
    transmission['Quarter_Hour'] = transmission['Weather Datetime'].dt.floor('15min')  

    grouped = transmission.groupby(['Run Name', 'Year', 'Quarter_Hour'])  # Group by 15-minute intervals

    for (run_name, year, quarter_hour), group in grouped:
        # Create lookup dictionary for flows
        flow_dict = dict(zip(zip(group['Source'], group['Destination']), group['Value']))

        processed_pairs = set()

        for (zone1, zone2), value1 in flow_dict.items():
            if (zone2, zone1) in flow_dict:
                if (zone1, zone2) not in processed_pairs:
                    value2 = flow_dict[(zone2, zone1)]
                    if value1 > value2:
                        net_flow_data.append((zone1, zone2, run_name, year, quarter_hour, value1 - value2))
                    elif value2 > value1:
                        net_flow_data.append((zone2, zone1, run_name, year, quarter_hour, value2 - value1))
                    processed_pairs.add((zone1, zone2))
                    processed_pairs.add((zone2, zone1))
            elif (zone1, zone2) not in processed_pairs:
                net_flow_data.append((zone1, zone2, run_name, year, quarter_hour, value1))
                processed_pairs.add((zone1, zone2))

    net_flow_df = pd.DataFrame(net_flow_data, columns=['Source', 'Destination', 'Run Name', 'Year', 'Weather Datetime', 'Value'])
    return net_flow_df



transmission[['Source', 'Destination']] = transmission.apply(extract_zones, axis=1)

transmission_filled = fill_missing_hours(transmission)

transmission_interpolated = interpolate_transmission(transmission_filled)

net_transmission = compute_net_flows(transmission_interpolated)

In [ ]:
def interpolate_adp(adp_data, type_col_labels):
    df = adp_data.copy()  # Avoid modifying the original DataFrame
    df['Weather Datetime'] = pd.to_datetime(df['Weather Datetime'])  # Ensure datetime format
    df['date'] = df['Weather Datetime'].dt.date  # Extract date (day-level grouping)
    
    # Create the combination of columns to check for duplicates
    group_columns = ['Weather Datetime', 'Zone', type_col_labels] if isinstance(type_col_labels, list) else ['Weather Datetime', 'Zone', type_col_labels]
    
    # Check for duplicates based on the combination of 'Weather Datetime', 'Zone', and type_col_labels
    duplicate_timestamps = df[df.duplicated(subset=group_columns, keep=False)]
    
    df = df.set_index('Weather Datetime')  # Set index for resampling

    interpolated_results = []  # List to store results

    for zone, group in df.groupby(['Zone', 'date', type_col_labels]): 
        # Drop duplicates within the group (if any)
        group = group[~group.index.duplicated(keep='first')]

        group = group.infer_objects(copy=False)
        
        group_resampled = group.resample(f'{interpolate_minutes}min').interpolate(method='linear')  # Interpolate numeric columns
        
        # Forward-fill non-numeric columns
        for col in group.select_dtypes(exclude=['number']).columns:
            group_resampled[col] = group_resampled[col].ffill()
        
        group_resampled = group_resampled.reset_index()  # Reset index for merging later
        interpolated_results.append(group_resampled)  # Store result

    interpolated_df = pd.concat(interpolated_results, ignore_index=True)
    return interpolated_df


adp_to_filtered = adp_data_to[adp_data_to["From||Type"] != 'transmission'].fillna({'Value': 0})
adp_from_filtered = adp_data_from[adp_data_from["To||Type"] != 'transmission'].fillna({'Value': 0})

adp_to_interp = interpolate_adp(adp_to_filtered, "From||Outputs Group Aggregate")
adp_from_interp = interpolate_adp(adp_from_filtered, "To||Outputs Group Aggregate")

In [ ]:
arrow_color = "#4682b4"
pie_multiplier = 1/200

color_mapping_to = {
    'electricity storage': '#9b59b6',  # Soft purple
    'gas power': '#d2691e',  # Soft brown
    'onshore wind': '#2e8b57',  # Soft green
    'solar': '#f4c542',  # Soft yellow
    'canadian imports': '#4682b4',  # Soft blue
    'gas power w/cc': '#ff8c00',  # Soft orange
    'geothermal power': '#87cefa',  # Light blue
    'hydro': '#00bcd4',  # Soft cyan
    'nuclear power': '#808080',  # Soft grey
    'offshore wind': '#1e3a8a',  # Dark blue
    'oil power': '#2f2f2f'  # Dark grey (almost black)
}

color_mapping_elec_use = {
    'boiler': '#1f77b4',  # Blue
    'commercial': '#ff7f0e',  # Orange
    'data center': '#2ca02c',  # Green
    'electricity storage': '#d62728',  # Red
    'electrolysis h2': '#9467bd',  # Purple
    'productive': '#8c564b',  # Brown
    'residential': '#e377c2',  # Pink
    'thermal energy storage': '#7f7f7f',  # Gray
    'transportation': '#bcbd22',  # Olive
    'upstream losses': '#17becf',  # Cyan
    'biofuels': '#aec7e8',  # Light Blue
    'haber-bosch': '#ffbb78',  # Light Orange
    'hydrogen liquefaction': '#98df8a',  # Light Green
    'iron and steel production': '#ff9896',  # Light Red
    'lng production': '#c5b0d5'  # Light Purple
}

In [ ]:


def compute_terminator_coordinates(input_datetime, nlons=360):
    """
    Compute day-night terminator coordinates for a given datetime.

    Args:
        input_datetime (str or datetime): The date and time in UTC (e.g., '2011-01-29 13:00:00').
        nlons (int): Number of longitudes to compute (default is 360).

    Returns:
        pd.DataFrame or None: DataFrame with columns ['Longitude', 'Latitude'] containing terminator coordinates,
                              or None if no coordinates are within the USA latitude range.
    """
    dg2rad = np.pi / 180.0
    est = timezone(timedelta(hours=-5))
    
    # Convert input to UTC datetime
    if isinstance(input_datetime, str):
        input_datetime = datetime.strptime(input_datetime, '%Y-%m-%d %H:%M:%S')
    date_est = input_datetime.replace(tzinfo=est)  
    date_utc = date_est.astimezone(timezone.utc)  

    # Set up the observer
    observer = ephem.Observer()
    observer.date = ephem.Date(date_utc)

    # Compute solar declination and hour angle
    sun = ephem.Sun(observer)
    sun.compute(observer)

    dec = np.degrees(sun.dec)  # Solar declination in degrees
    tau = np.degrees(observer.sidereal_time() - sun.ra)  # Hour angle in degrees

    # Generate longitudes for the continental USA
    lons = np.linspace(-125, -67.5, nlons)
    longitude = lons + tau  # Adjust longitudes based on hour angle

    # Compute latitudes
    lats = np.arctan(-np.cos(longitude * dg2rad) / np.tan(dec * dg2rad)) / dg2rad

    # Create a DataFrame
    df = pd.DataFrame({'Longitude': lons, 'Latitude': lats})

    # Filter latitudes to within the continental USA range (24.5° to 49.5°)
    df = df[(df['Latitude'] >= 24.5) & (df['Latitude'] <= 49.5)]

    # Return None if no valid coordinates exist
    return df if not df.empty else None


In [ ]:



def energy_animation(date, shapefile_path, transmission, adp_interp, lat_long_info, color_mapping, pie_multiplier, arrow_color, frames_per_second, save_path, chart_type, interpolate_minutes, scenario):
    # Load US map and filter for the continental US
    interval_ms = 100
    us_map = gpd.read_file(shapefile_path)
    us_map = us_map.to_crs(epsg=4326)

    # Set up figure and axis
    fig, ax = plt.subplots(figsize=(15, 10))
    us_map.plot(ax=ax, color='white', edgecolor='black')

    # Global storage for arrows and pies
    arrow_patches = []
    pie_axes = []

    if chart_type not in ["to", "from"]:
        raise ValueError("chart_type must be either 'to' or 'from'")

    type_col_labels = "From||Outputs Group Aggregate" if chart_type == "to" else "To||Outputs Group Aggregate"

    adp_interp = adp_interp[adp_interp['Run Name'] == scenario]
    transmission = transmission[transmission['Run Name'] == scenario]

    def update(frame):
        nonlocal arrow_patches, pie_axes
        global terminator_line  # Declare a variable to track the terminator line

        # Clear previous arrows and pie charts
        for arrow in arrow_patches:
            arrow.remove()
        for pie_ax in pie_axes:
            pie_ax.remove()

        arrow_patches.clear()
        pie_axes.clear()

        if "terminator_line" in globals() and terminator_line is not None:
            terminator_line.remove()
            terminator_line = None

        # Define input datetime for current frame
        input_datetime = pd.Timestamp(date) + pd.Timedelta(minutes=interpolate_minutes * frame)

        # Filter transmission data
        filtered_transmission = transmission[transmission['Weather Datetime'] == input_datetime].dropna(subset=['Value'])

        # Compute pie chart sizes
        pie_sizes = {}
        filtered_adp = adp_interp[adp_interp['Weather Datetime'] == input_datetime]
        grouped_by_zone = filtered_adp.groupby('Zone')

        for zone, group in grouped_by_zone:
            total_value = group['Value'].sum()
            pie_sizes[zone] = total_value * pie_multiplier

        # Draw transmission arrows
        for _, row in filtered_transmission.iterrows():
            source = lat_long_info[lat_long_info['zone'] == row['Source']]
            dest = lat_long_info[lat_long_info['zone'] == row['Destination']]

            if not source.empty and not dest.empty:
                full_line = LineString([(source.iloc[0]['longitude'], source.iloc[0]['latitude']),
                                        (dest.iloc[0]['longitude'], dest.iloc[0]['latitude'])])

                buffer_start = pie_sizes.get(row['Source'], 0) * 2
                buffer_end = pie_sizes.get(row['Destination'], 0) * 2

                start_point = full_line.interpolate(buffer_start)
                end_point = full_line.interpolate(full_line.length - buffer_end)

                new_line = LineString([start_point, end_point])

                value = row['Value'] / 7

                arrow = patches.FancyArrowPatch((new_line.coords[0][0], new_line.coords[0][1]),
                                                (new_line.coords[1][0], new_line.coords[1][1]),
                                                mutation_scale=10,
                                                color=arrow_color,
                                                linewidth=value)
                ax.add_patch(arrow)
                arrow_patches.append(arrow)

        # Draw pie charts
        for zone, group in grouped_by_zone:
            sizes = group['Value'].values
            labels = group[type_col_labels].values

            point = lat_long_info[lat_long_info['zone'] == zone]
            if not point.empty:
                lat, lon = point.iloc[0]['latitude'], point.iloc[0]['longitude']
                if -125 <= lon <= -66.5 and 24.5 <= lat <= 49.5:
                    size = pie_sizes.get(zone, 0)
                    x, y = ax.transData.transform((lon, lat))
                    pie_ax = inset_axes(ax, width=size, height=size, loc='center', bbox_to_anchor=(x, y))
                    pie_colors = [color_mapping.get(label, 'gray') for label in labels]
                    pie_ax.pie(sizes, autopct='', startangle=90, wedgeprops={'linewidth': 0.1, 'edgecolor': 'black'}, colors=pie_colors)
                    pie_axes.append(pie_ax)

        # Plot the solar terminator
        terminator_coords = compute_terminator_coordinates(input_datetime)
        if terminator_coords is not None and not terminator_coords.empty:
            terminator_line, = ax.plot(terminator_coords['Longitude'], terminator_coords['Latitude'], color="orange", linewidth=5, label="Solar Terminator")
        
        
        # Title
        if chart_type == "to":
            title = f"Electricity Generation: {input_datetime.strftime('%m-%d %H:%M')}"
        else:
            title = f"Load: {input_datetime.strftime('%m-%d %H:%M')}"
        ax.set_title(title)

    # Create legends
    legend_GWs = [10, 50, 100]
    legend_marker_sizes = [1/4 * GW for GW in legend_GWs]
    pie_size_patches = [Line2D([0], [0], marker='o', color='grey', markerfacecolor='gray',
                               markersize=legend_marker_sizes[i], label=f'{legend_GWs[i]} GW', linewidth=0)
                        for i in range(len(legend_GWs))]
    
    pie_legend = ax.legend(handles=pie_size_patches, 
                           loc='lower left', 
                           fontsize=12, 
                           frameon=False,     
                           handleheight=1, 
                           handlelength=2,
                           labelspacing=1.5,
    )

    # Transmission line size legend
    transmission_GWs = [10, 20, 30]  # Example GW values

    transmission_patches = [
    Line2D([0], [0], color=arrow_color, linewidth=transmission_GWs[i]/4, 
        label=f'{transmission_GWs[i]} GW') for i in range(len(transmission_GWs))
    ]

    transmission_legend = ax.legend(handles=transmission_patches, 
                            loc='lower left', 
                            fontsize=12, 
                            frameon=False, 
                            handleheight=1,
                            handlelength=2,
                            bbox_to_anchor=(0.15, 0.02))  # Adjust position
    
    source_legend_patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label)
                              for label, color in color_mapping.items()]
    source_legend = plt.legend(handles=source_legend_patches, loc='lower right', frameon=False)
    ax.add_artist(pie_legend)
    ax.add_artist(transmission_legend)  
    ax.set_axis_off()

    # Animate through all 24 hours
    #frames = 24 * 4 - 3
    frames = [52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92]

    anim = animation.FuncAnimation(fig, update, frames=frames, interval=interval_ms, blit=False)

    # Save animation
    filename = f"{save_path}/animation_{chart_type}_{date}.mp4"
    FFwriter = animation.FFMpegWriter(fps=frames_per_second)
    anim.save(filename, writer=FFwriter)
    plt.close()

    print(f"Animation saved: {filename}")


In [ ]:
#energy_animation(date, shapefile_path, net_transmission, adp_from_interp, lat_long_info, color_mapping_elec_use, pie_multiplier, arrow_color, frames_per_second, save_path, "from", interpolate_minutes, scenario)
energy_animation(date, shapefile_path, net_transmission, adp_to_interp, lat_long_info, color_mapping_to, pie_multiplier, arrow_color, frames_per_second, save_path, "to", interpolate_minutes, scenario)